# Groq + E2B MCP Gateway: Multiple MCP Servers in One Sandbox

This notebook demonstrates how to use E2B's MCP Gateway to run multiple Model Context Protocol servers together with Groq's ultra-fast inference.

## What is E2B MCP Gateway?

E2B's MCP Gateway provides a unified interface to multiple MCP servers running inside an isolated sandbox. Instead of connecting to each MCP server separately, you get:

- **One endpoint** for all your MCP tools (Exa, Browserbase, Notion, etc.)
- **Isolated environment** where MCPs run securely
- **Seamless integration** with Groq's Responses API
- **200+ tools** from the Docker MCP Catalog

## What We'll Build

We'll create an AI research agent that:
1. **Searches** for AI companies using Exa MCP's web search
2. **Visits** their websites using Browserbase MCP's browser automation
3. **Extracts** pricing and contact information
4. **Returns** structured results—all through one E2B gateway

This showcases how E2B makes it trivial to combine multiple MCP capabilities in a single workflow.


## Getting Started

You'll need API keys from four services:

1. **E2B** ([e2b.dev](https://e2b.dev/)) - Provides the unified MCP gateway and sandbox
2. **Browserbase** ([browserbase.com](https://www.browserbase.com/)) - Browser automation MCP
   - You'll need both an API key AND a Project ID
3. **Exa** ([dashboard.exa.ai](https://dashboard.exa.ai/api-keys)) - Web search MCP
4. **Groq** ([console.groq.com](https://console.groq.com/keys)) - Ultra-fast LLM inference

### Set Environment Variables

After obtaining your API keys, set them as environment variables:

**macOS/Linux:**
```bash
export E2B_API_KEY='your_e2b_key'
export BROWSERBASE_API_KEY='your_browserbase_key'
export BROWSERBASE_PROJECT_ID='your_project_id'
export EXA_API_KEY='your_exa_key'
export GROQ_API_KEY='your_groq_key'
```

**Windows (PowerShell):**
```powershell
$Env:E2B_API_KEY='your_e2b_key'
$Env:BROWSERBASE_API_KEY='your_browserbase_key'
$Env:BROWSERBASE_PROJECT_ID='your_project_id'
$Env:EXA_API_KEY='your_exa_key'
$Env:GROQ_API_KEY='your_groq_key'
```


## Installation

Install the required packages:


In [ ]:
%pip install e2b==2.4.0 openai

## Setup: Import Libraries and Configure API Keys


In [ ]:
import json
import os
import time
from datetime import datetime

from e2b import Sandbox
from openai import OpenAI

# Load API keys from environment
E2B_API_KEY = os.getenv("E2B_API_KEY")
BROWSERBASE_API_KEY = os.getenv("BROWSERBASE_API_KEY")
BROWSERBASE_PROJECT_ID = os.getenv("BROWSERBASE_PROJECT_ID")
EXA_API_KEY = os.getenv("EXA_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Validate API keys
missing_keys = []
if not E2B_API_KEY:
    missing_keys.append("E2B_API_KEY")
if not BROWSERBASE_API_KEY:
    missing_keys.append("BROWSERBASE_API_KEY")
if not BROWSERBASE_PROJECT_ID:
    missing_keys.append("BROWSERBASE_PROJECT_ID")
if not EXA_API_KEY:
    missing_keys.append("EXA_API_KEY")
if not GROQ_API_KEY:
    missing_keys.append("GROQ_API_KEY")

if missing_keys:
    print("❌ Missing required API keys:")
    for key in missing_keys:
        print(f"   - {key}")
    print("\nPlease set all required environment variables before continuing.")
else:
    print("✅ All API keys configured successfully!")
    print("   - E2B API key")
    print("   - Browserbase API key + Project ID")
    print("   - Exa API key")
    print("   - Groq API key")

# Model configuration
MODEL = "openai/gpt-oss-120b"


## Step 1: Create E2B Sandbox with MCP Gateway

Here's where the magic happens. We create a single E2B sandbox that hosts both Exa and Browserbase MCP servers:


In [ ]:
print("🚀 Creating E2B sandbox with MCP gateway...")
start = time.time()

# Create sandbox with multiple MCP servers configured
sandbox = Sandbox.create(
    api_key=E2B_API_KEY,
    mcp={
        'browserbase': {
            'apiKey': BROWSERBASE_API_KEY,
            'projectId': BROWSERBASE_PROJECT_ID,
        },
        'exa': {
            'apiKey': EXA_API_KEY,
        }
    },
    timeout=600
)

# Get the unified gateway URL and authentication token
mcp_url = sandbox.get_mcp_url()
mcp_token = sandbox.get_mcp_token()

elapsed = time.time() - start
print(f"✅ Sandbox created in {elapsed:.2f}s")
print(f"   Gateway URL: {mcp_url}")
print(f"   Sandbox Info: {sandbox.get_info()}")


## Step 2: Setup Groq Client

Initialize the Groq client using OpenAI's SDK (Groq is OpenAI-compatible):


In [ ]:
client = OpenAI(
    base_url="https://api.groq.com/api/openai/v1",
    api_key=GROQ_API_KEY
)

print("✅ Groq client initialized")


## Step 3: Configure MCP Tools for Groq

Tell Groq about the E2B MCP gateway. Notice we only need ONE tool configuration—E2B routes to all MCPs internally:


In [ ]:
tools = [
    {
        "type": "mcp",
        "server_url": mcp_url,
        "server_label": "e2b-gateway",
        "require_approval": "never",
        "headers": {
            "Authorization": f"Bearer {mcp_token}"
        }
    }
]

print("✅ MCP tools configured for Groq")
print(f"   Using unified E2B gateway at: {mcp_url}")


## Core Function: Research with Multiple MCPs

This function demonstrates the power of E2B's unified gateway:


In [ ]:
def research_companies_with_e2b(query, client, tools):
    """
    Research companies using multiple MCP servers through E2B's unified gateway.
    
    This function demonstrates:
    - Exa MCP for web search
    - Browserbase MCP for browser automation
    - All through one E2B gateway endpoint
    """
    
    print(f"\n🔍 Research Query: {query}")
    print("-" * 80)
    
    start_time = time.time()
    
    # Call Groq with E2B MCP gateway
    # The LLM will automatically use Exa and Browserbase as needed
    response = client.responses.create(
        model=MODEL,
        input=query,
        tools=tools,
        stream=False,
        temperature=0.1,
        top_p=0.4,
    )
    
    total_time = time.time() - start_time
    
    # Extract response content
    content = response.output_text if hasattr(response, "output_text") else str(response)
    
    # Collect executed MCP calls
    executed_tools = []
    exa_calls = 0
    browserbase_calls = 0
    
    if hasattr(response, "output") and response.output:
        for output_item in response.output:
            if hasattr(output_item, "type") and output_item.type == "mcp_call":
                tool_info = {
                    "type": "mcp",
                    "name": getattr(output_item, "name", ""),
                    "server_label": getattr(output_item, "server_label", ""),
                    "arguments": getattr(output_item, "arguments", "{}"),
                }
                executed_tools.append(tool_info)
                
                # Count calls by MCP server
                tool_name = tool_info["name"].lower()
                if "exa" in tool_name or "search" in tool_name:
                    exa_calls += 1
                elif "browserbase" in tool_name or "stagehand" in tool_name:
                    browserbase_calls += 1
    
    return {
        "content": content,
        "total_time": total_time,
        "mcp_calls_performed": executed_tools,
        "exa_calls": exa_calls,
        "browserbase_calls": browserbase_calls,
        "timestamp": datetime.now().isoformat(),
    }


def print_research_results(result):
    """
    Pretty-print research results showing which MCPs were used.
    """
    print("\n" + "=" * 80)
    print("RESEARCH RESULTS")
    print("=" * 80)
    print(result["content"])
    print("=" * 80)
    
    # Show MCP usage
    executed_tools = result["mcp_calls_performed"]
    if executed_tools:
        print(f"\n🔧 MCP CALLS: {len(executed_tools)} tool calls executed")
        print(f"   📡 Exa calls: {result['exa_calls']}")
        print(f"   🌐 Browserbase calls: {result['browserbase_calls']}")
        print("-" * 50)
        
        for i, tool in enumerate(executed_tools, 1):
            print(f"\n   Call #{i}: {tool['name']}")
            print(f"   Server: {tool['server_label']}")
            
            # Parse and display arguments
            try:
                args = json.loads(tool["arguments"]) if isinstance(tool["arguments"], str) else tool["arguments"]
                print(f"   Arguments: {json.dumps(args, indent=6)[:200]}...")
            except:
                print(f"   Arguments: {str(tool['arguments'])[:200]}...")
    
    print(f"\n⏱️  PERFORMANCE")
    print(f"   Total time: {result['total_time']:.2f}s")
    print(f"   MCP calls: {len(executed_tools)}")
    print("=" * 80)


## Demo 1: Research AI Inference Companies

Let's research AI companies that provide inference APIs and extract their pricing information.

Watch how the agent:
1. Uses **Exa** to search for AI inference companies
2. Uses **Browserbase** to visit their websites
3. Extracts pricing and contact details

All of this happens through the **single E2B gateway** we configured:


In [ ]:
result1 = research_companies_with_e2b(
    "Find 2 AI inference API companies and get their pricing information from their websites",
    client,
    tools
)

print_research_results(result1)


Display the response in markdown format:


In [ ]:
from IPython.display import Markdown

Markdown(result1["content"])


## Demo 2: Custom Research Task

Now it's your turn! Replace the query below with your own research task.

Ideas to try:
- "Find popular AI image generation tools and check their pricing"
- "Research vector database companies and extract their key features"
- "Find AI agent frameworks and get their documentation links"
- "Research LLM observability tools and compare their features"


In [ ]:
custom_query = "Your research query here"  # Change this!

result2 = research_companies_with_e2b(
    custom_query,
    client,
    tools
)

print_research_results(result2)


## Next Steps

Now that you've seen E2B's MCP gateway in action, here are some ideas to explore:

### Add More MCP Servers
E2B supports 200+ MCP servers from the Docker MCP Catalog. Try adding:
- **Notion** - Store research findings in a database
- **Airtable** - Organize data in spreadsheets
- **GitHub** - Search code repositories
- **Slack** - Send notifications
- **Firecrawl** - Advanced web scraping

Just add them to the `mcp` configuration:
```python
sandbox = Sandbox.create(
    mcp={
        'browserbase': {...},
        'exa': {...},
        'notion': {
            'internalIntegrationToken': NOTION_API_KEY,
        },
        # Add more...
    }
)
```

### Build Production Workflows
- Create scheduled research jobs that run daily
- Build a research API endpoint
- Set up monitoring and alerting
- Store results in a database

### Scale with Multiple Sandboxes
- Run parallel research tasks in separate sandboxes
- Implement rate limiting and queuing
- Add retry logic and error handling

### Experiment with Different Models
Try different Groq models for different tasks:
- `llama-3.3-70b-versatile` - Great for general tasks
- `qwen/qwen3-32b` - Fast and efficient
- `openai/gpt-oss-120b` - Powerful reasoning

### Optimize for Speed
- Use streaming responses for real-time updates
- Batch similar requests together
- Cache frequently accessed data

---


## Resources

### E2B MCP Gateway
- **Documentation**: [e2b.dev/docs/mcp](https://e2b.dev/docs/mcp)
- **Quickstart**: [e2b.dev/docs/mcp/quickstart](https://e2b.dev/docs/mcp/quickstart)
- **Available Servers**: [e2b.dev/docs/mcp/available-servers](https://e2b.dev/docs/mcp/available-servers)
- **Custom Servers**: [e2b.dev/docs/mcp/custom-servers](https://e2b.dev/docs/mcp/custom-servers)
- **GitHub**: [github.com/e2b-dev/e2b](https://github.com/e2b-dev/e2b)

### MCP Servers Used
- **Exa MCP**: [docs.exa.ai/reference/exa-mcp](https://docs.exa.ai/reference/exa-mcp)
- **Exa GitHub**: [github.com/exa-labs/exa-mcp-server](https://github.com/exa-labs/exa-mcp-server)
- **Browserbase MCP**: [smithery.ai/server/@browserbasehq/mcp-browserbase](https://smithery.ai/server/@browserbasehq/mcp-browserbase)
- **Stagehand Docs**: [docs.stagehand.dev](https://docs.stagehand.dev/)

### Groq
- **API Documentation**: [console.groq.com/docs](https://console.groq.com/docs)
- **Models**: [console.groq.com/docs/models](https://console.groq.com/docs/models)
- **Cookbook**: [github.com/groq/groq-api-cookbook](https://github.com/groq/groq-api-cookbook)

### Model Context Protocol
- **Official Site**: [modelcontextprotocol.io](https://modelcontextprotocol.io)
- **Specification**: [spec.modelcontextprotocol.io](https://spec.modelcontextprotocol.io)
- **GitHub**: [github.com/modelcontextprotocol](https://github.com/modelcontextprotocol)

---

**Happy researching with E2B + Groq! 🚀**
